In [1]:
data=sqlContext.read.format('csv')\
                    .option("header", "false")\
                    .option("inferschema", "true")\
                    .option("mode", "DROPMALFORMED")\
                    .load('data/kddcup.data.corrected')

(1)

In [2]:
import statistics

q1_data = data.select(['_c0', '_c4', '_c5', '_c10']).rdd.map(list).collect()
q1_data = sc.parallelize(q1_data)

mean = [q1_data.map(lambda x:x[i]).mean() for i in range(4)]
median = [statistics.median(q1_data.map(lambda x:x[i]).collect()) for i in range(4)]
mode = [statistics.mode(q1_data.map(lambda x:x[i]).collect()) for i in range(4)]
std = [q1_data.map(lambda x:x[i]).stdev() for i in range(4)]

print('header:', 'duration', 'src_bytes', 'dst_bytes', 'num_failed_logins')
print('mean:', mean)
print('median:', median)
print('mode:', mode)
print('std:', std)

header: duration src_bytes dst_bytes num_failed_logins
mean: [48.34243046395799, 1834.6211752293461, 1093.6228137132064, 3.2051079212915824e-05]
median: [0, 520, 0, 0]
mode: [0, 1032, 0, 0]
std: [723.3297374218567, 941430.9783959766, 645012.2679038465, 0.007299406830851142]


(2)

In [3]:
from operator import add

q2_data = data.select(['_c1', '_c2', '_c3', '_c11', '_c41']).rdd.map(list)

result = [q2_data.map(lambda x:(x[i], 1)).reduceByKey(add).sortBy(lambda x:x[1], False).collect() for i in range(5)]

header = ['protocol_type', 'service', 'flag', 'logged_in', 'intrusion_type']
for i, h in enumerate(header):
    print(h+str(':'))
    print(result[i], '\n')

protocol_type:
[('icmp', 2833545), ('tcp', 1870598), ('udp', 194288)] 

service:
[('ecr_i', 2811660), ('private', 1100831), ('http', 623091), ('smtp', 96554), ('other', 72653), ('domain_u', 57782), ('ftp_data', 40697), ('eco_i', 16338), ('finger', 6891), ('urp_i', 5378), ('ftp', 5214), ('telnet', 4277), ('ntp_u', 3833), ('auth', 3382), ('pop_3', 1981), ('time', 1579), ('domain', 1113), ('Z39_50', 1078), ('gopher', 1077), ('mtp', 1076), ('ssh', 1075), ('remote_job', 1073), ('whois', 1073), ('rje', 1070), ('link', 1069), ('imap4', 1069), ('ctf', 1068), ('name', 1067), ('supdup', 1060), ('echo', 1059), ('nntp', 1059), ('discard', 1059), ('uucp_path', 1057), ('daytime', 1056), ('netstat', 1056), ('sunrpc', 1056), ('systat', 1056), ('pop_2', 1055), ('netbios_ssn', 1055), ('netbios_ns', 1054), ('vmnet', 1053), ('sql_net', 1052), ('iso_tsap', 1052), ('netbios_dgm', 1052), ('csnet_ns', 1051), ('shell', 1051), ('hostnames', 1050), ('klogin', 1050), ('bgp', 1047), ('login', 1045), ('exec', 1045)

(3)

In [4]:
q3_data = data.select(['_c2', '_c41']).rdd.map(list)

intrusionType = q3_data.map(lambda x:x[1]).distinct().collect()
result = [q3_data.map(lambda x:((x[0], x[1]), 1)).reduceByKey(add).filter(lambda x:x[0][1] == intrusionType[i])\
.map(lambda x:(x[0][0], x[1])).sortBy(lambda x:x[1], False).collect() for i in range(len(intrusionType))]

for i, t in enumerate(intrusionType):
    print('intrusionType:', t)
    print(result[i])

intrusionType: loadmodule.
[('telnet', 5), ('ftp_data', 3), ('ftp', 1)]
intrusionType: neptune.
[('private', 1013720), ('telnet', 1923), ('ftp_data', 1805), ('http', 1801), ('finger', 1800), ('smtp', 1140), ('Z39_50', 1066), ('domain', 1046), ('pop_3', 1046), ('uucp_path', 1044), ('mtp', 1044), ('pop_2', 1043), ('sunrpc', 1043), ('nntp', 1043), ('imap4', 1043), ('supdup', 1042), ('netbios_ssn', 1042), ('whois', 1042), ('ftp', 1042), ('remote_job', 1041), ('vmnet', 1041), ('netbios_ns', 1041), ('discard', 1040), ('klogin', 1040), ('echo', 1040), ('time', 1040), ('ssh', 1039), ('gopher', 1039), ('netbios_dgm', 1039), ('sql_net', 1039), ('link', 1038), ('iso_tsap', 1038), ('csnet_ns', 1038), ('netstat', 1038), ('systat', 1038), ('rje', 1038), ('daytime', 1037), ('auth', 1037), ('ctf', 1037), ('hostnames', 1037), ('http_443', 1036), ('name', 1036), ('exec', 1035), ('bgp', 1035), ('printer', 1034), ('shell', 1034), ('ldap', 1033), ('login', 1032), ('efs', 1032), ('nnsp', 1030), ('kshell', 1

(4)

In [5]:
q4_data = data.select(['_c4', '_c10']).rdd.map(list)

q4_count = q4_data.count()

q4_mean_a = q4_data.map(lambda x:x[0]).mean()
q4_mean_b = q4_data.map(lambda x:x[1]).mean()

q4_std_a = q4_data.map(lambda x:x[0]).stdev()
q4_std_b = q4_data.map(lambda x:x[1]).stdev()

result = q4_data.map(lambda x:(x[0]-q4_mean_a)*(x[1]-q4_mean_b)).reduce(add)
result /= ((q4_count - 1)*q4_std_a*q4_std_b)
print(result)

-6.921907666018592e-06


(optional bonus)

In [7]:
q5_data = data.select(['_c15', '_c16', '_c17', '_c18', '_c19', '_c20', '_c21', '_c41']).rdd.map(list)

intrusionType = q5_data.map(lambda x:x[7]).distinct().collect()
value_label = ['same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',\
               'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate']

for i in range(len(intrusionType)):
    temp = sc.parallelize(q5_data.filter(lambda x:x[7] == intrusionType[i]).collect())
    print(intrusionType[i], ':')
    for j in range(len(value_label)):
        result = temp.map(lambda x:x[j]).sortBy(lambda x:x, False).take(1)[0]
        print('highest', value_label[j], 'value:', result)
    print()

loadmodule. :
highest same_srv_rate value: 3
highest diff_srv_rate value: 4
highest srv_diff_host_rate value: 2
highest dst_host_count value: 1
highest dst_host_srv_count value: 0
highest dst_host_same_srv_rate value: 0
highest dst_host_diff_srv_rate value: 0

neptune. :
highest same_srv_rate value: 0
highest diff_srv_rate value: 0
highest srv_diff_host_rate value: 0
highest dst_host_count value: 0
highest dst_host_srv_count value: 0
highest dst_host_same_srv_rate value: 0
highest dst_host_diff_srv_rate value: 0

guess_passwd. :
highest same_srv_rate value: 0
highest diff_srv_rate value: 0
highest srv_diff_host_rate value: 0
highest dst_host_count value: 0
highest dst_host_srv_count value: 0
highest dst_host_same_srv_rate value: 0
highest dst_host_diff_srv_rate value: 1

pod. :
highest same_srv_rate value: 0
highest diff_srv_rate value: 0
highest srv_diff_host_rate value: 0
highest dst_host_count value: 0
highest dst_host_srv_count value: 0
highest dst_host_same_srv_rate value: 0
highe